<center><h1>Algorithmique Avancée</h1>
        <h2>Livrable du projet</h2></center>
        
<h3><u>Introduction</u> :</h3>
<h4><u>Contexte</u> :</h4>

Notre équipe de CesiCDP, spécialisée dans la mobilité et la logistique du transport, a souhaité répondre à l’appel de l’ADEME en proposant des solutions de gestion de tournées de livraison. Le problème consiste à calculer une tournée reliant un sous-ensemble de villes en revenant au point de départ qui minimise le temps de trajet. </br>
Dans ce livrable, nous vous présentons le problème, le reformulons de manière formelle et étudions sa complexité. Nous vous présentons également la méthode de résolution choisie, en vous détaillant l'algorithme utilisé.

<h4><u>Présentation de notre équipe</u> :</h4>

<table style="float:left;width:85%">
    <tr><td style="text-align:left;width:18%">
<strong>Alexandre GUYOT</strong>
        </td><td style="text-align:left;width:15%">
        Albin GANDNER
        </td><td style="text-align:left;width:22%">
        Yoann <br> MIGLIONICO-JACQUOT
        </td><td style="text-align:left;width:20%">
        Sébastien BAEUMLIN
        </td><td style="text-align:left;width:15%">
        Abbé N'KPOMIN
        </td></tr>
    <tr><td>
<img src="Images/Alexandre.jpg" style="width:100px"/>
        </td><td>
<img src="Images/Albin.jpg" style="width:100px"/>
        </td><td>
<img src="Images/Yoann.jpg" style="width:100px"/>
        </td><td>
<img src="Images/Sébastien.jpg" style="width:100px"/>
        </td><td>
<img src="Images/Abbé.jpg" style="width:100px"/>
        </td></tr>
</table>

<h3>1. <u>Modélisation du problème</u> :</h3>
<h4><u>Représentation formelle du problème</u> :</h4>

<p style="text-align:justify">Notre projet correspond à un problème de gestion des tournées de livraison. Un véhicule part d’une ville et doit réaliser un parcours en passant au moins une fois par chaque ville du sous-ensemble, puis revenir au point de départ, en minimisant la durée totale de la tournée.</p>

<p style="text-align:justify">Nous pouvons représenter ce problème par un graphe non orienté, où chaque ville est représentée par un sommet et chaque route empruntée pour aller d’une ville à une autre par une arête-valué, où le poids représente la distance à parcourir. On considère que la distance à parcourir nous donne une estimation du temps de trajet.</p>

<strong>Données : </strong>
Un graphe $G=(V,E)$ avec :
<ul style="text-align:justify">
    <li>Un ensemble fini $V$ de sommets représentation des villes.</li>
    <li>Un ensemble fini $E=${ $(i,j)$ $|i,j \in V, i \neq j$ } de toutes les paires non ordonnées $(i,j)$, où $i$ et $j$ sont des éléments de $V$. Cet ensemble contient toutes les combinaisons possibles de paires de villes connectées par des arêtes dans le graphe $G$.</li>
    <li>Un ensemble de variables $d_{ij}$ représentant la longueur de chaque arête $(i,j) \in E$, c'est-à-dire la distance séparant le ville $i$ de la ville $j$ (avec $i,j \in V$)</li></ul>

<strong>Variables de décision du programme : </strong>
<ul style="text-align:justify">
    <li>$P[i]$ : Liste des sommets à visiter ($i \in \mathbb{N}^+, i \leq n$).</li>
    <li>$D_t$ : Distance totale du trajet, calculée en faisant la somme des poids des arêtes traversées. On note $\sum d_{ij}$, avec $d_{ij}\in \mathbb{R}^+$ le poids de l’arête entre deux sommets $i$ et $j$.</li></ul>

<strong>Contraintes : </strong>

<ul style="text-align:justify">
    <li><u>Chaque ville doit être visitée au moins une fois.</u> $\forall i,j\in \mathbb{N}^+,i,j<n,i\neq j$, on a $\sum A[i][j] \geq 1$ la somme de tous les sommets $i$, où $A[i][j]\in $ {$0,1$} est une matrice binaire de taille $n\times n$ définit comme suit : $1$ si le sommet $j$ est visité après le sommet $i$, $0$ sinon. Cette contrainte stipule que pour chaque sommet $i$, la somme des éléments de la ligne $i$ de la matrice $A$ doit être supérieure ou égale à $1$. Cela garantit que chaque sommet du graphe est visitée au moins une fois, car si la somme des éléments de la ligne $i$ est supérieure ou égale à $1$, cela implique qu'il existe au moins un sommet $j$ qui est visité après le sommet $i$, donc chaque sommet du graphe sera visité au moins une fois. </li>
    <li><u>Les villes successives de la liste doivent être reliées.</u> $\forall i \in \{0,1,...,n-2\}:(P[i] \in V)\land(P[i+1] \in V) \rightarrow (P[i], P[i+1]) \in E$, où $V$ est l'ensemble des sommets du graphe et $E$ l'ensemble des arêtes du graphe. Cela présice que pour chaque indice $i$ de la liste $P$, si les sommets $P[i]$ et $P[i+1]$ sont tous les deux à visiter, alors il doit y avoir une arête entre ces deux sommets dans le graphe.</li>
    <li><u>La tournée doit commencer et se terminer dans la même ville.</u> $P[0]=P[n]$, où $P[0]$ correspond au premier sommet de la liste des sommets visités et $P[n]$ correspond  au dernier.</li>
    <li><u>Chaque ville dispose d’une fenêtre de temps de livraison à respecter.</u> $t_{i_{min}} \leq t_i \leq t_{i_{max}}$, avec $t_i \in \mathbb{R}^+$ le temps à l’arrivée au sommet $i$, $t_{i_{min}} \in \mathbb{R}^+$ et $t_{i_{max}} \in \mathbb{R}^+$ le temps minimum et maximum dans notre fenêtre de temps.</li>
    <li><u>Le temps de parcours d’une arête varie au cours du temps, représentant la variation du trafic.</u> $d_{ij} = d_{ij} + K$, avec $d_{ij} \in \mathbb{R}^+$ le poids de l’arête entre deux sommets $i$ et $j$ et $K \in \mathbb{R}$ une variable aléatoire définit comme suit : $K = [d_{ij} \times -\frac{5}{100} ; d_{ij}\times \frac{5}{100}]$.</li>
    <li><u>Les poids des arêtes ne doivent pas être des valeurs négatives.</u> $d_{ij} \geq 0$.</li>
</ul>

<strong>Fonction objectif : </strong>

<blockquote>Minimiser la durée totale de la tournée. $\text{Minimiser }\sum(d_{ij} \times A[i][j] \times k)$ pour toutes les arêtes ($i$, $j$) visitées, où $d_{ij} \in \mathbb{R}^+$ est le poids des arêtes entre les villes $i$ et $j$, $A[i][j]\in \{0,1\}$ la matrice binaire de taille $n\times n$ indiquant si le sommet $j$ est visité après le sommet $i$, nous permettant de savoir quelles arêtes sont empruntées et $k \in \mathbb{N}^+$ est le nombre de fois que l’arête est empruntée.</blockquote>
 	
<strong>Programme complet : </strong>
<blockquote>
$\text{Minimiser }\sum(d_{ij} \times A[i][j] \times k), \forall i,j\in \mathbb{N}^+,i,j \leq n \\
\text{s.c.	}\forall i,j\in \mathbb{N}^+,i,j \leq n,i\neq j, \sum A[i][j] \geq 1 \\
    \forall i \in \{0,1,...,n-2\}:(P[i] \in V)\land(P[i+1] \in V) \rightarrow (P[i], P[i+1]) \in E \\
	P[0] = P[n] \\
	t_{i_{min}} \leq t_i \leq t_{i_{max}} | t_{i_{min}},t_i,t_{i_{max}} \in \mathbb{R} \\
	d_{ij} = d_{ij} +  [d_{ij} \times -\frac{5}{100} ; d_{ij}\times \frac{5}{100}], d_{ij} \in \mathbb{R}^+ \\
    d_{ij} \geq 0$
</blockquote>



<h3>2. <u>Démonstration de la complexité théorique du problème</u> :</h3>

<p style="text-align:justify">Trouver la classe de complexité de notre problème nous permet de le classer en fonction de sa difficulté intrinsèque et de l’efficacité de l’algorithme qui le résout.</p>

<p style="text-align:justify">La classe NP regroupe les problèmes pour lesquels une solution peut être vérifiée en temps polynomial mais pour lesquels la recherche d’une solution elle-même n’est pas garantie en temps polynomial.
La classe NP-difficile regroupe les problèmes qui sont au moins aussi difficiles que tous les autres problèmes de la classe NP.
La classe NP-complet regroupe les problèmes les plus difficiles de NP. C’est une sous-classe de NP-difficile.</p>

<p style="text-align:justify">Pour déterminer la classe de notre problème, nous allons tenter de prouver que notre problème est NP-complet. Pour ce faire, nous devons chercher s’il appartient à la classe NP et NP-difficile.</p>


<h4><u>Dans le cas où le problème est une optimisation</u> :</h4>

<p style="text-align:justify">Un problème appartient à la classe NP s'il peut être vérifié en temps polynomial. Cela signifie que pour chaque instance du problème (dans notre cas, une tournée spécifique), il existe un certificat (ou une preuve) qui peut être vérifié en temps polynomial pour déterminer si la solution proposée est valide ou non.</p>

    
<p style="text-align:justify">Dans le cas de notre problème, la solution proposée est une tournée spécifique qui visite chaque ville au minimum une fois et revient au point de départ, en minimisant la distance totale parcourue.</p>

<p style="text-align:justify">Pour vérifier si une tournée proposée est valide, nous devons effectuer les vérifications suivantes :</p>


<ul style="text-align:justify">
    <li><u>Vérifier si chaque ville est visitée au minimum une fois</u> : Cela peut être fait en parcourant la tournée et en vérifiant si chaque ville apparaît au moins une fois dans la séquence. La complexité de cette vérification est de l'ordre de $O(n)$, où n est le nombre de villes dans la tournée. </li>
    <li><u>Vérifier que les villes successives de la liste sont connectées</u> : Nous devons vérifier si chaque ville de la liste a une arête sortante vers la ville suivante dans la tournée. Si le voisin du sommet $i$ n'est pas le sommet $j$, cela signifie qu'il n'est pas possible d'y accéder depuis la ville $i$ et donc la solution proposée n'est pas valide. La complexité de cette vérification est de l’ordre de $O(n)$ car il faut effectuer cette vérification sur tous les éléments de la liste.</li>
    <li><u>Vérifier si le trajet est une boucle</u> : Cela peut être fait en comparant le premier et le dernier élément de la liste pour voir s'ils sont égaux. La complexité de cette vérification est de l'ordre de $O(1)$. </li>
    <li><u>Vérifier que la fenêtre de temps est respectée lors de la traversée de chaque sommet</u> : Cela peut être fait en comparant le temps pris par le véhicule pour accéder au sommet, ajouté au temps de départ, par la fenêtre de temps du sommet en question. La complexité de cette vérification est de l'ordre de $O(n)$, car la vérification est effectuée à chaque ville de la liste. </li>
    <li><u>Déterminer la longueur totale la plus courte parmi l’intégralité des tournées possibles</u> : Pour garantir de trouver la tournée la plus courte, il est nécessaire de déterminer la longueur de toutes les tournées possibles pour les comparer. Plus il y a de sommets à visiter, plus le nombre de trajets possibles augmente. Dans le pire des cas (où le graphe est complet), chaque sommet du graphe a $n-1$ chemins possibles à vérifier. Ainsi la complexité de cet algorithme est de l’ordre de $O(n!)$.</li>
</ul>

<p style="text-align:justify">Toutes ces vérifications ne se font pas en temps polynomial, nous avons la détermination de la longueur la plus courte qui n'est réalisable qu'en temps factoriel $O(n!)$. Le temps nécessaire pour vérifier si une solution est valide n’augmente donc pas de manière polynomiale avec la taille de l'instance du problème.</p>

<p style="text-align:justify">Ainsi, dans le cas où notre problème est un problème d’optimisation, il n’appartient pas à NP.</p>

<br>

<p style="text-align:justify">Pour montrer que notre problème est NP-difficile, il faut qu’on choisisse un problème NP-difficile connu et qu'on le réduise polynomialement à notre problème. La réduction polynomiale est une transformation d'un problème en un autre problème, où la transformation est réalisée en temps polynomial, à l’aide d’une fonction de réduction. Cette fonction transforme une instance d’un problème A en une instance de B, de telle sorte que la réponse à l'instance de A est oui si et seulement si la réponse à l'instance correspondante de B est oui.</p>

<p style="text-align:justify">Dans notre cas, si l’on réduit polynomialement le problème que l’on sait NP-difficile en notre problème de tournée de livraison et que l’instance ainsi obtenue est vérifiée, alors notre problème est au moins aussi difficile que le problème connu comme NP-difficile et par extension démontrer que notre problème est NP-difficile.</p>

<p style="text-align:justify">Prenons le problème du Voyageur de Commerce avec Fenêtre de Temps (TSPTW). Dans ce problème, nous cherchons à trouver le chemin le plus court permettant de visiter l’ensemble des villes avant de revenir à la ville de départ, en sachant que chaque ville est associée à une fenêtre de temps spécifique pendant laquelle elle peut être visitée. Ce problème est connu comme NP-difficile.</p>

<p style="text-align:justify">Nous devons montrer que ce problème se réduit polynomialement au problème de tournée de livraison.</p>

<p style="text-align:justify">Considérons un algorithme qui prend en entrée une instance $I_{vc}$ du problème Voyageur de Commerce avec Fenêtre de Temps, constitué du graphe $G=(V,E)$ avec $V$ représentant l'ensemble des sommets de $G$, avec une fenêtre de temps à chaqu'un deux et $E$ représente l'ensemble des arêtes de $G$. Cet algorithme retourne l’instance $I_{tl}$ du problème de tournée de livraison constituée du graphe $G’=(V',E')$ où chaque sommet de l'ensemble $V'$ correspond aux sommets de l'ensemble $V$ avec une fenêtre de temps pplus large, pour nous assurer que la contrainte est vérifiée quel que soit le temps pris pour accéder aux sommets, où l'ensemble $E'=E(G)+E(\bar{G})$, avec $\bar{G})$ le conjugué du graphe $G$, ce qui nous donne un graphe complet et où les poids des arêtes de l'ensemble $E'$ sont de $1$ pour celles traversées par la solution de l'instance $I_{vc}$ et sont $\infty$ pour les arêtes non traversées par la solution.</p>

<p style="text-align:justify">Si nous avons une solution à l’instance $I_{tl}$ avec notre algorithme, cela implique qu’il existe une solution à l'instance $I_{vc}$ avec l'algorithme TSPTW. S'il n'existe pas de solution à l'instance $I_{tl} avec notre algorithme, alors il n'exite pas de solution à l'instance $I_{vc}$ du problème TSPTW.</p>

<p style="text-align:justify">Par conséquent, nous avons établi une réduction polynomiale du problème TSPTW à notre problème. Comme TSPTW est NP-difficile, cela implique que notre problème est NP-difficile.</p>

<p style="text-align:justify">Le problème de la tournée de livraison est donc NP-difficile car on sait que TSPTW est NP-difficile. Cela veut dire que le problème est difficile à résoudre de manière générale et donc qu'il faut trouver des compromis pour trouver une solution satisfaisante.</p>


<h4><u>Dans le cas où le problème est décisionnel</u> :</h4>

<p style="text-align:justify">Trouver la solution optimale à notre problème va prendre un temps considérable. Nous pouvons tenter de relier le problème d’optimisation à un problème de décision pour étudier sa faisabilité plutôt que de trouver la solution optimale. Cela nous permettra également d’utiliser des techniques de résolutions spécifiquement conçus pour les problèmes de décision.</p>

<p style="text-align:justify">Dans le cadre de notre problème d’optimisation, nous devons déterminer le chemin le plus court. Pour que ce problème devienne un problème décisionnel, nous devons définir une contrainte de décision $c\in ℝ^+$, donc qui aura pour réponse “oui” ou “non”. Dans notre cas, nous allons vérifier si la distance totale de la tournée (notre variable de décision) est inférieure à une limite imposée. Cette transformation va nous permettre de rendre l’algorithme résolvable en un temps polynomial.</p>

<blockquote> Question : Existe-t-il un circuit passant au moins une fois par chaque sommet de G dans la fenêtre de temps et dont la somme des valeurs des arêtes est au plus c ?</blockquote>

<p style="text-align:justify">Vérifions si ce nouveau problème de décision est dans la classe des NP : </p>

<ul style="text-align:justify">
    <li><u>Vérifier si chaque ville est visitée au minimum une fois</u> :  La complexité de cette vérification a déjà été montrée comme étant de l'ordre de $O(n)$, où n est le nombre de villes dans la tournée.</li>
    <li><u>Vérifier que tous les sommets sont reliés</u> : La complexité de cette vérification a aussi été démontrée comme étant de l’ordre de $O(n)$.</li>
    <li><u>Vérifier que le trajet est une boucle</u> : La complexité de cette vérification a été démontrée comme étant de l’ordre de $O(1)$.</li>
    <li><u>Vérifier que la fenêtre de temps est respectée lors de la traversée de chaque sommet</u> : La complexité de cette vérification a été démontrée comme étant de l’ordre de $O(n)$.</li>
    <li><u>Vérifier si la longueur totale de la tournée est inférieure à une certaine valeur</u> : Nous devons calculer la distance totale parcourue dans la tournée et vérifier si elle est inférieure à une limite donnée. Cela garantit que la solution proposée respecte la contrainte de minimisation de la distance totale. La complexité de cette vérification dépend du nombre d'arêtes dans la tournée, donc la complexité est de $O(n)$.</li>
</ul>

<p style="text-align:justify">Ces vérifications peuvent être effectuées en temps polynomial, c'est-à-dire que le temps nécessaire pour vérifier si une solution est valide augmente proportionnellement avec la taille de l'instance du problème. Notre problème est donc dans la classe NP.</p>

<br>

<p style="text-align:justify">Notre problème a déjà démontré comme NP-difficile. L'ajout de la contrainte de décision ne simplifie pas la recherche de solution du problème. Donc, comme notre problème de tournée de livraison est dans NP et qu'il est NP-difficile, il est NP-Complet.</p>

<h3>3. <u>Méthode de résolution</u> :</h3>

<p style="text-align:justify">Nous considérons notre problème comme étant décisionnel en ajoutant la contrainte de décision $c \in ℝ^+$ comme donnée d'entrée du problème.</p>

<p style="text-align:justify">Ainsi nous devons déterminer comment obtenir un chemin le plus court possible passant par toutes les villes du graphe dans les fenêtres de livraison tout en ayant un temps de recherche de solution le plus faible possible. Pour ce faire nous allons utiliser une métaheuristique.</p>

<p style="text-align:justify">Une métaheuristique est un algorithme d’optimisation visant à résoudre des problèmes de décision difficiles pour lesquels on ne connaît pas de méthode classique plus efficaces.</p>

<h4><u>Colonie de fourmis</u> :</h4>

<p style="text-align:justify">Notre première proposition de métaheuristique est la colonie de fourmis. Comme son nom l'indique, elle s’inspire du comportement social des fourmis. En marchant du nid à la source de nourriture, les fourmis déposent au passage sur le sol une substance odorante appelée phéromones. Cette substance permet de créer une piste chimique sur laquelle les fourmis s’y retrouvent. Les phéromones ont un rôle de marqueur de chemin : quand les fourmis choisissent leur chemin,
elles ont tendance à choisir la piste qui porte la plus forte concentration de phéromones. Cela leur permet de retrouver le chemin vers leur nid lors du retour. D’autre part, les odeurs peuvent etre utilisées par les autres fourmis pour retrouver les sources de nourritures trouvées par leurs congénères.</p>

<p style="text-align:justify">Ce comportement permet de trouver le chemin le plus court vers la nourriture lorsque les pistes
de phéromones sont utilisées par la colonie entière. Autrement dit, lorsque plusieurs chemins marqués sont à la disposition d’une fourmi, cette dernière peut connaitre le chemin le plus court vers sa destination.</p>

<p style="text-align:justify">Pour vous présentez leur comportement, je vais vous parler de l'expérience du pont binaire de Deneubourg.</p>

<img src="Images/Deneubourg.jpg" style="width:800px"/>

<p style="text-align:justify">Au départ, il n’y a pas de phéromone sur le pont. Donc, chaque branche peut être choisie par une fourmi avec la même probabilité. Néanmoins, dans notre exemple, après une certaine période, des variations aléatoires ont fait qu’un peu plus de fourmis ont choisi le chemin du haut plutôt que celui du bas. Puisque les fourmis déposent des phéromones en avançant et puisqu’elles sont plus nombreuses en haut qu’en bas, le chemin du haut comportera plus de phéromones. Cette quantité supérieure de phéromone incite plus de fourmis à choisir la branche du haut, donc la quantité de phéromone déposée augmentera encore plus. On en déduit que plus les fourmis suivent un chemin, plus ce chemin devient interessant à suivre. Ainsi la probabilité avec laquelle une fourmi choisit un chemin, augmente avec le nombre de fourmis qui ont pris ce chemin précédemment.</p>

<p style="text-align:justify">Il est intéressant de remarquer que bien qu’une seule fourmi soit capable de construire une solution, c’est seulement le comportement de l’ensemble de la colonie qui crée le chemin le plus court.</p>

<p style="text-align:justify">Pour ce qui est des paramètres de notre algorythme, nous avons donc :</p>

<p style="text-align:justify">
Une matrice de villes : L'algorithme des fourmis utilise un graphe pour représenter le problème à résoudre. Chaque nœud du graphe représente une ville ou un emplacement potentiel, et les arêtes du graphe représentent les connexions entre ces emplacements.

Taux d'évaporation : Les phéromones déposées sur les arêtes s'évaporent avec le temps pour éviter une accumulation excessive. Le taux d'évaporation contrôlent la vitesse à laquelle les phéromones s'évaporent. Des valeurs plus élevées conduisent à une évaporation plus rapide.

Intensification : L'intensification fait référence au renforcement des phéromones sur les chemins empruntés par les meilleures fourmis. Ce paramètre régule l'importance relative de cet aspect dans la mise à jour des phéromones.

Nombre de fourmis : Le nombre de fourmis utilisées dans l'algorithme influe sur l'exploration de l'espace des solutions. Un nombre plus élevé de fourmis augmente l'exploration, mais peut également augmenter le temps de calcul.

Alpha : Ce paramètre contrôle l'influence des phéromones déposées sur les arêtes du graphe. Une valeur plus élevée d'alpha signifie que les fourmis seront davantage influencées par les niveaux de phéromones lorsqu'elles prennent une décision de mouvement. Une valeur élevée d'alpha favorise l'exploitation des chemins riches en phéromones, ce qui peut conduire à une convergence plus rapide vers une solution globalement bonne.

Beta : Ce paramètre régule l'influence de l'heuristique locale lors du choix du prochain mouvement. L'heuristique est généralement basée sur des informations locales, telles que la distance entre les villes ou la qualité de la solution partielle actuelle. Une valeur plus élevée de beta donne plus de poids à l'heuristique lors du choix du prochain mouvement, ce qui favorise l'exploration de nouvelles régions du graphe. Une valeur plus élevée de beta encourage l'exploration et peut aider à éviter de rester coincé dans des solutions locales sous-optimales.
</p>

optimizer = AntColonyOptimizer(len(matrix), ants=10, evaporation_rate=0.5, intensification=0.8, alpha=1, beta=2)

<p style="text-align:justify">Pour vous donner un exemple de la différence entre une valeur élevé d'alpha et une valeur élevé de beta, voici une comparaison entre deux graphique avec graph de 100 villes avec 100 itérations et 10 fourmies avec un graph avec alpha à 1 et beta à 10 et un graph avec alpha à 10 et beta à 1.</p>

<img src="Images/alpha10.png" style="width:1200px"/>

<img src="Images/beta10.png" style="width:1200px"/>

<p style="text-align:justify">Nous remarquons bien qu'avec un alpha élevé, nous allons suivre les phéromones et resté dans un optimum local (nous aurons donc une suite de valeur plus stable mais nous n'obtiendront pas la meilleur valeur) alors qu'avec un beta élevé, nous allons rechercher un optimum global (nous avons une meilleure valeur plus faible mais chaque itération est plus disparâtre).</p>

<p style="text-align:justify">Voici l'algorithme : </p>

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import time
import random
import warnings

class AntColonyOptimizer:
    def __init__(self, matrix, ants, evaporation_rate, intensification, alpha, beta, choose_best=1):
        """
        Constructeur et initialisateur du programme
        """     
        self.ants = ants
        self.evaporation_rate = evaporation_rate
        self.pheromone_intensification = intensification
        self.heuristic_alpha = alpha
        self.heuristic_beta = beta
        self.choose_best = choose_best

        self.pheromone_matrix = None
        self.heuristic_matrix = None
        self.probability_matrix = None
        self.mapDebut = None
        self.map = None
        self.set_of_available_nodes = None

        self.best_series = []
        self.best = None
        self.fitted = False
        self.best_path = None
        self.fit_time = None
        self.value = setValue(matrix)

        self.stopped_early = False

    def _reinstate_nodes(self):
        """
        Initialise le modèle en créant les différentes matrices et en générant la liste des nœuds disponibles.
        """
        self.set_of_available_nodes = list(range(self.map.shape[0]))

    def _remove_node(self, node):
        self.set_of_available_nodes.remove(node)

    def _initialize(self):
        """
        Après évaporation et intensification, la matrice de probabilité doit être mise à jour. Cette fonction s'en charge.
        """
        assert self.map.shape[0] == self.map.shape[1], "Map is not a distance matrix!"
        num_nodes = self.map.shape[0]
        self.pheromone_matrix = np.ones((num_nodes, num_nodes))

        self.pheromone_matrix[np.eye(num_nodes) == 1] = 0
        self.heuristic_matrix = 1 / self.map
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)  
        self.set_of_available_nodes = list(range(num_nodes))


    def _update_probabilities(self):
        """
        After evaporation and intensification, the probability matrix needs to be updated.  This function
        does that.
        """
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)

    def _choose_next_node(self, from_node):
        """
        Choisit le prochain nœud en fonction des probabilités. Si p < p_choisir_meilleur, alors le meilleur chemin est choisi, sinon il est sélectionné à partir d'une distribution de probabilité pondérée par la phéromone.
        """
        numerator = self.probability_matrix[from_node, self.set_of_available_nodes]
        if np.random.random() < self.choose_best:
            next_node = np.argmax(numerator)
        else:
            denominator = np.sum(numerator)
            probabilities = numerator / denominator
            next_node = np.random.choice(range(len(probabilities)), p=probabilities)
        return next_node

    def _evaluate(self, paths):
        """
        Évalue les solutions en additionnant les distances entre les nœuds.
        """
        scores = np.zeros(len(paths))
        coordinates_i = []
        coordinates_j = []
        for index, path in enumerate(paths):
            score = 0
            coords_i = []
            coords_j = []
            for i in range(len(path) - 1):
                coords_i.append(path[i])
                coords_j.append(path[i + 1])
                score += self.map[path[i], path[i + 1]]
            scores[index] = score
            coordinates_i.append(coords_i)
            coordinates_j.append(coords_j)
        best = np.argmin(scores)
        return (coordinates_i[best], coordinates_j[best]), paths[best], scores[best]

    def _evaporate(self):
        """
        Multiplie la matrice de pheromone par l'inverse du taux d'evaporation 
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)

    def _intensify(self, best_coords):
        """
        Incrémente la matrice de pheromone
        """
        i = best_coords[0]
        j = best_coords[1]
        self.pheromone_matrix[i, j] += self.pheromone_intensification

    def fit(self, map_matrix, iterations):
        """
        Lancement des itérations de l'ACO
        """
        print("Lancement de l'ACO pour {} iterations :".format(iterations))

        self.mapDebut = map_matrix
        self.map = changeMatrix(self.mapDebut)
        start = time.time()
        self._initialize()
        num_equal = 0

        for i in range(iterations):
            start_iter = time.time()
            paths = []
            path = []

            for ant in range(self.ants):
                current_node = self.set_of_available_nodes[np.random.randint(0, len(self.set_of_available_nodes))]
                start_node = current_node
                while True:
                    path.append(current_node)
                    self._remove_node(current_node)
                    if len(self.set_of_available_nodes) != 0:
                        current_node_index = self._choose_next_node(current_node)
                        current_node = self.set_of_available_nodes[current_node_index]
                    else:
                        break

                path.append(start_node)  
                self._reinstate_nodes()
                paths.append(path)
                path = []

            best_path_coords, best_path, best_score = self._evaluate(paths)

            if i == 0:
                best_score_so_far = best_score
            else:
                if best_score < best_score_so_far:
                    best_score_so_far = best_score
                    self.best_path = best_path

            if best_score == best_score_so_far:
                num_equal += 1
            else:
                num_equal = 0

            self.best_series.append(best_score)
            self._evaporate()
            self._intensify(best_path_coords)
            self._update_probabilities()

            self.map = changeMatrix(self.mapDebut)

            print("Meilleur valeur de l'itération {}: {} - Meilleur valeur globale: {} ({}s)".format(i, round(best_score * self.value), round(best_score_so_far * self.value),round((time.time() - start_iter), 3)))

        self.fit_time = time.time() - start
        self.fitted = True

        self.best = self.best_series[np.argmin(self.best_series)]
        print("ACO finis. Temps d'éxécution: {} secondes. Meilleur valeur globale : {}".format(round(self.fit_time,3), round(self.best * self.value)))
        return self.best

    def plot(self):
        """
        Affichage du graphique pour comparer les itérations entre elles
        """
        fig, ax = plt.subplots(figsize=(20, 15))
        ax.plot(self.best_series, label="Best run")
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Performance")
        ax.text(.8, .6,
                'Number Ants: {}\Evap rate: {}\nIntensification: {}\nAlpha: {}\nBeta: {}\nBeta Evap: {}\n\nTime : {}m{}'.format(
                    self.ants, self.evaporation_rate, self.pheromone_intensification, self.heuristic_alpha,
                    self.heuristic_beta, self.evaporation_rate, round(self.fit_time, 3),
                    ["\nStopped Early!" if self.stopped_early else ""][0]),
                bbox={'facecolor': 'gray', 'alpha': 0.8, 'pad': 10}, transform=ax.transAxes)
        ax.legend()
        plt.title("ACO (Best result: {})".format(np.round(self.best * self.value)))
        plt.show()

def plot_animated_route(matrix, path):
    """
    Affichage du graphique animé pour représenter l'animations de la meilleurs routes parcourant les villes (TEST)
    """
    fig, ax = plt.subplots()
    ax.set_title("Animation de la meilleure route")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    nodes = np.arange(matrix.shape[0])
    coordinates = np.random.rand(matrix.shape[0], 2)

    ax.scatter(coordinates[:, 0], coordinates[:, 1], color='blue', label="Villes")

    line, = ax.plot([], [], color='red', lw=2, label="Meilleure route")

    def update(frame):
        current_edge = (path[frame], path[frame + 1])

        x = [coordinates[current_edge[0], 0], coordinates[current_edge[1], 0]]
        y = [coordinates[current_edge[0], 1], coordinates[current_edge[1], 1]]

        line.set_data(x, y)
        return line,

    ani = anim.FuncAnimation(fig, update, frames=len(path) - 1, interval=1000, repeat=True)
    ax.legend()

    plt.show()

def createMatrix(size):
    """
    Créer une matrice adjacente simétrique pour simulé notre ville
    """
    matrix = np.random.randint(1, 100, size=(size, size))
    matrix = np.triu(matrix, k=1)
    matrix += matrix.T
    np.fill_diagonal(matrix, 0)
    return matrix

def changeMatrix(matrix):
    """
    Change les valeurs de la matrice de plus ou moins 5%
    """
    matrix = matrix.astype(float)
    change = np.random.uniform(low=-0.05, high=0.05, size=matrix.shape)
    matrix += matrix * change
    return matrix

def create_time_window_matrix(matrix, time_windows):
    """
    Créer une matrice de fenêtre de temps
    """
    new_matrix = matrix.copy()
    for i in range(len(time_windows)):
        for j in range(len(time_windows)):
            if i != j:
                new_matrix[i, j] *= time_windows[j]
    return new_matrix

random.seed(0)
warnings.filterwarnings("ignore")

matrix = createMatrix(100)

time_windows = np.random.uniform(0.5, 2, size=(matrix.shape[0],))
time_window_matrix = create_time_window_matrix(matrix, time_windows)

optimizer = AntColonyOptimizer(len(matrix), ants=10, evaporation_rate=0.5, intensification=0.8, alpha=10, beta=1)
best = optimizer.fit(time_window_matrix, iterations = 100)
optimizer.plot()

best_path = optimizer.best_path
plot_animated_route(matrix, best_path)

<h4><u>Recuit simulé</u> :</h4>

<p style="text-align:justify">Notre deuxième proposition de métaheuristique est le recuit simulé. Elle est inspiré du processus de recuit utilisé dans la métallurgie pour améliorer la qualité des matériaux. En partant du matériau étudié, on effectue des modifications pour changer ses caractéristiques et relaxer des contraintes. </p>

<p style="text-align:justify"> En algorithmique avancée, on part d'une configuration initiale du problème à résoudre et on génère, à chaque itération, un solution voisine en apportant des modifications aléatoire à la solution courante. La solution voisine est évaluée par une fonction objectif pour mesurer sa qualité. Une solution de plus faible qualité peut, de manière aléatoire en utilisant une fonction de probabilité, être acceptée, évitant ainsi de se retrouver dans un optimum local. Le processus de refroidissement vise quand à lui à réduire progressivement la probabilité d'accepter des solutions de qualité inférieure au fil du temps, pour se diriger vers un optimum global.</p>

<p style="text-align:justify">Nous avons trois valeur d'entrée en plus de la matrice :</p>

<p style="text-align:justify">Cooling_rate : il s'agit d'une valeur entre 0 et 1, à chaque itération du programme, celle-ci sera multipliée à la température, diminuant celle-ci par la même occasion.</p>

<p style="text-align:justify">Initial_temperature: il s'agit de la température initiale. La température tout au long de l'exécution de notre programme sera amené à diminuer puisque  chaque itération celle-ci se verra multipliée par le cooling_rate. Plus la température initiale est élevée et plus le programme acceptera des solutions supérieures à la solution de l'itération de précédente</p>

<p style="text-align:justify">Max_iteration : cette valeur définit le nombre d'itérations que le programme va réaliser. Plus la valeur sera élevée et plus le résultat va tendre vers un optimum.</p>

<p style="text-align:justify"> Maintenant changeons les valeurs initials de Max_iteration et de Initial_temperature.</p>
<p style="text-align:justify"> Commençons par modifier la valeur Max_itération respectivement de 50/150/500. La valeur de Initial_temperature est de 1000 pour les 3 graphes et le cooling_rate est de 0.98</p>

<table>
<tr><td>
<img src="Images/1000_50.png" style="width:800px"/>
    </td><td>
<img src="Images/1000_150.png" style="width:800px"/>
    </td><td>
<img src="Images/1000_500.png" style="width:800px"/>
    </td>
</tr>
    </table>
<p style="text-align:justify"> On remarque plusieurs choses :</p>
<p style="text-align:justify"> Si le nombre d'itérations est trop faible, nous n'aboutissons pas à une valeur suffisamment faible et le programme continue d'accepter des solutions supérieures à la valeur la plus faible déjà trouvée.</p>
<p style="text-align:justify"> Plus le nombre d'itérations est élevé, plus la température baisse et plus le programme va refuser des valeurs supérieures au coût le plus faible déjà trouvé, ainsi le résultat est de plus en plus faible.</p>
<p style="text-align:justify"> Maintenant modifions la valeurs de Initial_temperature avec respectivement 100/1000/10000. La valeur de Max_iteration est de 200 et le cooling_rate de 0.98.</p>

<table>
<tr><td>
<img src="Images/100_200.png" style="width:800px"/>
    </td><td>
<img src="Images/1000_200.png" style="width:800px"/>
    </td><td>
<img src="Images/10000_200.png" style="width:800px"/>
    </td>
</tr>
    </table>

<p style="text-align:justify">Nous observons à nouveau des différences. Plus la valeur Initial_Temperature est élevée, plus le programme va explorer des solutions ayant un coût supérieur aux solutions déjà trouvées précédemment.</p>

<p style="text-align:justify"> Voici l'algorithme, celui-ci ne fonctionne pas sur jupyter pour des raisons inconnues mais fonctionne parfaitement sur Visual Studio.</p>

import random
import numpy as np
import time

import matplotlib.pyplot as plt
random.seed(0)
def createMatrix(size):
    distance_matrix = [[0 for i in range(size)] for j in range(size)]
    time_windows = [[0 for i in range(2)] for j in range(size)]

    for i in range(size):
        for j in range(i + 1, size):
            if random.randint(1, 100) < 95:
                distance_matrix[i][j] = random.randint(1, 100)
                distance_matrix[j][i] = distance_matrix[i][j]

    # Generate random time windows for each customer
    for i in range(1, size):
        time_windows[i][0] = random.randint(0, 10)
        time_windows[i][1] = random.randint(time_windows[i][0] + 1, 20)

    return np.array(distance_matrix), time_windows

def generate_initial_solution(num_customers):
    solution = [i for i in range(1, num_customers)]
    random.shuffle(solution)
    solution.insert(0, 0)
    solution.append(0)  # Add depot as the last customer
    return solution

def solve_vrptw_with_simulated_annealing(distance_matrix, time_windows, initial_temperature, cooling_rate, max_iterations):
    num_customers = distance_matrix.shape[0]
    best_solution = generate_initial_solution(num_customers)
    best_cost = calculate_cost(best_solution, distance_matrix, time_windows)

    current_solution = best_solution
    current_cost = best_cost
    cost_history = []

    temperature = initial_temperature

    for iteration in range(max_iterations):
        distance_matrix = update_matrix(distance_matrix)
        neighbour_solution = generate_neighbour(current_solution)
        neighbour_cost = calculate_cost(neighbour_solution, distance_matrix, time_windows)

        cost_difference = neighbour_cost - current_cost

        if cost_difference < 0 or random.random() < np.exp(-cost_difference / temperature):
            current_solution = neighbour_solution
            current_cost = neighbour_cost
            
            if neighbour_cost < best_cost:
                best_solution = neighbour_solution
                best_cost = neighbour_cost
        cost_history.append(current_cost)
        temperature *= cooling_rate
        
    plt.plot(range(max_iterations), cost_history)
    plt.xlabel('Iterations')
    plt.ylabel('Current Cost')
    plt.title('Cost at Each Iteration')
    plt.show()

    return best_solution, best_cost, current_solution   
            
    

def calculate_cost(solution, distance_matrix, time_windows):
    num_customers = len(solution)
    cost = 0
    current_time = 0

    for i in range(num_customers - 1):
        customer1 = solution[i]
        customer2 = solution[i + 1]
        cost += distance_matrix[customer1][customer2]
        arrival_time = current_time + distance_matrix[customer1][customer2]
        wait_time = max(time_windows[customer2][0] - arrival_time, 0)
        current_time = arrival_time + wait_time + time_windows[customer2][1]
        cost += wait_time

    return cost

def generate_neighbour(solution):
    neighbour = solution.copy()
    customer1, customer2 = random.sample(range(1, len(solution) - 1), 2)
    neighbour[customer1], neighbour[customer2] = neighbour[customer2], neighbour[customer1]
    return neighbour



def update_matrix(distance_matrix):
    distance_matrix = distance_matrix.astype(float)
    change = np.random.uniform(low=-0.05, high=0.05, size=distance_matrix.shape)
    distance_matrix += distance_matrix * change
    return distance_matrix


def main():
    distance_matrix, time_windows = createMatrix(1000)

    initial_temperature = 10000
    cooling_rate = 0.98
    max_iterations = 200

    start_time = time.process_time()
    current_solution,best_solution, best_cost = solve_vrptw_with_simulated_annealing(distance_matrix, time_windows, initial_temperature, cooling_rate, max_iterations)
    execution_time = time.process_time() - start_time

    
    print("Meilleure solution trouvée:")
    print(best_solution)
    print("Coût de la meilleure solution:")
    print(best_cost)
    print("Temps d'exécution: %.2f secondes" % execution_time)
    
    

if __name__ == "__main__":
    main()

    
<h3>4. <u>Choix de la meilleure méthode</u> :</h3>

<p style="text-align:justify">Chaque métaheursitque permet d'approcher la meilleur solution, la différence majeur entre elles qui va déterminer qu'elle métaheuristique est la plus approprié est l'implémentation.

Pour notre problème, l'algorythme de fourmis est bien plus facile à adapter. En effet, l'algorythme est idéal pour faire face au chagement de valeurs des arrêtes. Si la valeur de l'arrête est grandement augmenté, les phéromones vont s'évaporer et les fourmies vont donc emprunter un nouveau chemin. Dans notre cas, cela permettra d'éviter les ralentissement ainsi que les routes barrées.</p>
